In [1]:
import os, sys, shutil, subprocess, glob
path = os.getcwd()

seqtypes  = ['std','c05a','c15a','c20a','c30a']
incons    = ['oring','pcirc']
sequences = ["col"+str(i).zfill(2) for i in range(1, 11)]
dirs      = ['opt_log','opt_out','opt_par','opt_pdb','opt_refframe','opt_topology']

In [2]:
opttop = os.listdir(path+'/opt_topology')
opttop = [i.split('.')[0].lstrip('topo_') for i in opttop]
print(len(opttop))

7743


In [3]:
def rf_to_topology(initial_condition, sequence_length):
    infile = open(initial_condition, 'r')
    indat  = infile.readlines()
    infile.close()
    outfile = open(initial_condition, 'w')
    if len(indat) == 5*sequence_length+1:
        outfile.writelines([i for i in indat])
        circle = indat[1:6]
        outfile.writelines([i for i in circle])
    elif len(indat) == 5*sequence_length+6:
        outfile.writelines([i for i in indat[:-5]])
    outfile.close()
    return


In [4]:
for filename in os.listdir(path+'/opt_refframe'):
    name = filename.split('.')[0]
    if name not in opttop:
        os.chdir('opt_refframe')
        # Add first bp to last bp
        rf_to_topology(name+".dat", 150)
        # run emDNA_topology
        subprocess.run("emDNA_topology --x3DNA-bp-input="+name+".dat>topo_"+name+".txt --twist-density --virtual-last-bp", shell=True)
        shutil.move(path+"/opt_refframe/topo_"+name+".txt", path)
        # remove last bp
        rf_to_topology(name+".dat", 150)
        os.chdir(os.pardir)